In [246]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import edward2 as ed
tfd = tfp.distributions
import scipy.stats as st
from mpl_toolkits.mplot3d import Axes3D
import math

import seaborn as sns
import pandas as pd
%matplotlib inline


#### negative log of the posterior
\begin{equation}
\left(\frac{1}{2}\left\|\mathbf{f}(\mathrm{m})-\mathbf{d}_{\mathrm{obs}}\right\|_{\mathrm{\Gamma}_{\mathrm{noise}}^{-1}}^{2}+\frac{1}{2}\left\|\mathrm{m}-\mathrm{m}_{\mathrm{prior}}\right\|_{C_{\mathrm{prior}}^{-1}}^{2}\right)
\end{equation}


In [367]:
D = 2 # dimention
c = 0 # constant
N = 2 # number of data
sigma2y = 1 


std = 1

## a random draw from N~(theta2+theta1^2,1)
@tf.function
def f(theta1,theta2):
    k = tf.add(theta2,tf.pow(theta1,2.))
#     x = tf.random.normal([],mean = k,stddev = std)
#     return tf.convert_to_tensor(x,dtype=tf.float32)
    return k

prior = tf.convert_to_tensor([0.,0.])
cov = tf.convert_to_tensor([[1.,0.],[0.,1.]])

# mvn = tfd.MultivariateNormalFullCovariance(loc=prior, covariance_matrix=cov)

@tf.function
def matrixcompute(matrix1,matrix2,Cov):

    matrix = tf.subtract(matrix1, matrix2)
    matrix = tf.reshape(matrix,[matrix.shape[0],1])
    matrix_T = tf.transpose(matrix)
    Cov_inv = tf.linalg.inv(Cov)
    result = tf.multiply(tf.constant(1/2),tf.matmul(tf.matmul(matrix_T,Cov_inv),matrix))
    return result


y = np.random.normal(loc =c, scale = sigma2y, size = N) ## obervation data

sq_sum_y = np.sum(y**2)
mean_y = np.mean(y)

# y = tf.convert_to_tensor(y,dtype = tf.float32)
# y =  tf.data.Dataset.from_tensor_slices(y)

@tf.function
def log_post(theta1, theta2):
    ## Calculate the negative log posterior
    Sum = 
    ## Negative log likelihood
    log_likelihood = (1/2)* Sum/(std**2)
    mu=tf.convert_to_tensor([theta1,theta2])
    mu = tf.reshape(mu,[2,])
    ## Negative log prior
    log_prior = matrixcompute(mu, prior, cov)
    log_posterior = log_likelihood+log_prior
    
    return(log_posterior)


SyntaxError: invalid syntax (<ipython-input-367-f5943a3eab95>, line 44)

<b> find the MAP point by Gradient decent<b>

In [389]:
D = 2 # dimention
c = 0 # constant
N = 100 # number of data
sigma2y = 1 


std = 1

## a random draw from N~(theta2+theta1^2,1)
@tf.function
def f(theta1,theta2):
    k = tf.add(theta2,tf.pow(theta1,2.))
#     x = tf.random.normal([],mean = k,stddev = std)
#     return tf.convert_to_tensor(x,dtype=tf.float32)
    return k

lr = 0.1  
y = np.random.normal(loc =c, scale = sigma2y, size = N) ## obervation data
x = tf.constant([10.,10.])
for i in range(100):
    with tf.GradientTape() as t:  
        t.watch(x)
     
        Sum =  (y[0]-f(x[0],x[1]))**2+ (y[1]-f(x[0],x[1]))**2+(y[2]-f(x[0],x[1]))**2
                 
        dx = t.gradient(Sum,x)
        x = x-lr*dx
        print(x)


tf.Tensor([-1312.5513     -56.127563], shape=(2,), dtype=float32)
tf.Tensor([ 2.7134121e+09 -1.0336971e+06], shape=(2,), dtype=float32)
tf.Tensor([-2.3973339e+28 -4.4175634e+18], shape=(2,), dtype=float32)
tf.Tensor([ inf -inf], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], shape=(2,), dtype=float32)
tf.Tensor([nan nan], sha

In [315]:
def loss(theta1,theta2):
    return (log_post(theta1,theta2))

def loss_minimize():
    return (log_post(theta1,theta2))

opt = tf.keras.optimizers.SGD(learning_rate=0.1)

step = 500

In [316]:
dtype = np.float32



true_mean = [0, 0]
true_cov = [[1, 0],
                 [0, 1]]
num_results = 1000
num_chains = 1


target = tfd.MultivariateNormalFullCovariance(loc=true_mean, covariance_matrix=true_cov)

# Assume that the state is passed as a list of 1-d tensors `x` and `y`.
# Then the target log-density is defined as follows:
def target_log_prob(x, y):
  # Stack the input tensors together
  z = tf.stack([x, x**2+y], axis=-1)
  return target.log_prob(tf.squeeze(z))

# Initial state of the chain
init_state = [-3.,
              -2.]

# Run Random Walk Metropolis with normal proposal for `num_results`
# iterations for `num_chains` independent chains:
samples, kernel_results = tfp.mcmc.sample_chain(
    num_results=num_results,
    current_state=init_state,
    kernel=tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=target_log_prob,
        step_size = 0.1,
        num_leapfrog_steps = 5),
    num_burnin_steps=500,
    num_steps_between_results=1,  # Thinning.
    parallel_iterations=1)
samples = tf.stack(samples, axis=-1)

accepted = kernel_results.is_accepted

samples = samples.numpy()
accepted = accepted.numpy()

[97.694595]
[3.7889543] [-5.1314926]
[14095.973]
[-11.188242] [-7.107921]
[7.318266e+10]
[520.1472] [-30.853182]
[1.0035646e+31]
[-56284180.] [-54135.44]
[inf]
[7.132126e+22] [-6.3358176e+14]
[nan]
[-inf] [-inf]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [n

[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
[nan]
[nan] [nan]
